In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.6 MB/s eta 0:00:00


In [54]:
import torch
import os
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split, Dataset

from tqdm import tqdm
from process_data import build_train_test_loaders, build_train_test_loaders_2
from training import CustomMAELoss, CustomMAPELoss, test_model, get_flow_forecasting_metrics
from link_loads import get_graph_attributes, df_to_graph, build_quarter_hour_data, add_missing_nodes, add_missing_nodes_2
from models import STGCN
# import networkx as nx

In [18]:
folder_path = "/content/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

In [55]:
MAE = CustomMAELoss()
MAPE = CustomMAPELoss()

In [20]:
window_size = 4

In [21]:
dfs

{'Link_Loads_NBT16MTT.csv':                              Link             Line Dir  Order  From NLC  \
 0     ELEu_BAK_NB>LAMu_BAK_NB@BAK         Bakerloo  NB      1       570   
 1     LAMu_BAK_NB>WLOu_BAK_NB@BAK         Bakerloo  NB      2       628   
 2     WLOu_BAK_NB>EMBu_BAK_NB@BAK         Bakerloo  NB      3       747   
 3     EMBu_BAK_NB>CHXu_BAK_NB@BAK         Bakerloo  NB      4       542   
 4     CHXu_BAK_NB>PICu_BAK_NB@BAK         Bakerloo  NB      5       718   
 ...                           ...              ...  ..    ...       ...   
 1158  PIMu_VIC_SB>VUXu_VIC_SB@VIC         Victoria  SB     13       776   
 1159  VUXu_VIC_SB>STKu_VIC_SB@VIC         Victoria  SB     14       777   
 1160  STKu_VIC_SB>BRXu_VIC_SB@VIC         Victoria  SB     15       716   
 1161  WLOu_WAC_EB>BNKu_WAC_EB@WAC  Waterloo & City  EB      1       747   
 1162  BNKu_WAC_WB>WLOu_WAC_WB@WAC  Waterloo & City  WB      1       513   
 
      From ASC          From Station  To NLC To ASC        

In [22]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8)

### Test 1

In [ ]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn = GCNConv(node_features, hidden_dim)  # GCN
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim, 4)  # Prédiction finale
        self.fc2 = nn.Linear(4 + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)

        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features), dim=-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [ ]:
model = ImprovedSTGCN(node_features=window_size, hidden_dim=16)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 40

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1, Train loss: 48.95288218941953


  2%|▎         | 1/40 [00:11<07:47, 11.98s/it]

Epoch 1, Test loss: 47.144519521130455
Epoch 2, Train loss: 23.777809886203872


  5%|▌         | 2/40 [00:23<07:27, 11.78s/it]

...

Epoch 38, Test loss: 15.262246692464466
Epoch 39, Train loss: 14.21717159277863


 98%|█████████▊| 39/40 [08:02<00:12, 12.15s/it]

Epoch 39, Test loss: 14.976269902691008
Epoch 40, Train loss: 14.184178984562555


100%|██████████| 40/40 [08:13<00:00, 12.34s/it]

Epoch 40, Test loss: 14.836333154212861


In [ ]:
test_model(model, test_loader)

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 14.836332321166992
MAPE for high targets: 0.04389444959261233
MAE for low targets: 5.5507153447658295


### Test 2

In [ ]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim + node_features, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)

        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [ ]:
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 500

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train loss: 42.767816162904104


  0%|          | 1/500 [00:17<2:21:41, 17.04s/it]

Epoch 1, Test loss: 25.983634329031386
Epoch 2, Train loss: 28.558133171863027


  0%|          | 2/500 [00:33<2:17:38, 16.58s/it]

...

Epoch 498, Test loss: 12.757928732841734
Epoch 499, Train loss: 12.431549349427224


100%|█████████▉| 499/500 [1:01:38<00:07,  7.10s/it]

Epoch 499, Test loss: 12.757928732841734
Epoch 500, Train loss: 12.431549349427224


100%|██████████| 500/500 [1:01:46<00:00,  7.41s/it]

Epoch 500, Test loss: 12.757928732841734


In [ ]:
test_model(model, test_loader)

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 12.757927894592285
MAPE for high targets: 0.038881105119510304
MAE for low targets: 4.805690102633976


### Test 3

In [ ]:
class DeepSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(DeepSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.gcn3 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 8)  # Prédiction finale
        self.fc2 = nn.Linear(8 + temporal_features_dim + node_features, 16)
        self.fc3 = nn.Linear(16 + node_features, 8)
        self.fc4 = nn.Linear(8 + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.gcn3(x, graph.edge_index)  # GCN
        x = F.tanh(x) # (num_nodes, hideen_dim)
        x = self.fc1(x)
        x = F.tanh(x)
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.tanh(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc3(x)
        x = F.tanh(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc4(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [ ]:
model = DeepSTGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 500

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train loss: 42.802639022138386


  0%|          | 1/500 [00:23<3:15:14, 23.48s/it]

Epoch 1, Test loss: 22.921436099779037
Epoch 2, Train loss: 28.965290353298187


  0%|          | 2/500 [00:46<3:14:25, 23.42s/it]

...

Epoch 153, Test loss: 13.518963012430403
Epoch 154, Train loss: 13.331790215969086


 31%|███       | 154/500 [59:15<2:19:01, 24.11s/it]

Epoch 154, Test loss: 13.51892079815032


 31%|███       | 154/500 [59:32<2:13:47, 23.20s/it]


KeyboardInterrupt: 

In [ ]:
test_model(model, test_loader)

Test MAE: 13.51826000213623
MAPE for high targets: 0.04106026636214838
MAE for low targets: 5.213425626357396


In [ ]:
test_model(model, train_loader)

Test MAE: 13.328251838684082
MAPE for high targets: nan
MAE for low targets: 5.288397481242816


### Test 4

Modifs par rapport au meilleur modèle (Test 2): suffle les batchs

In [60]:
window_size = 4

In [61]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8, batch_size=32, shuffle=True)

In [62]:
i=0
for graph, temporal_features in train_loader:
  i+=1
print(i)

57


### Test 1 GPU batch size = 8

In [45]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + temporal_features_dim + node_features, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)

        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, temporal_features, graph.x), dim=-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [48]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 250

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        temporal_features = temporal_features.to(device, non_blocking=True)
        temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            temporal_features = temporal_features.to(device, non_blocking=True)
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1, Train loss: 35.02877683003744


  0%|          | 1/250 [00:01<06:14,  1.50s/it]

Epoch 1, Test loss: 21.914756517561656
Epoch 2, Train loss: 18.169094056023493


...

 53%|█████▎    | 132/250 [03:29<03:00,  1.53s/it]

Epoch 132, Test loss: 11.539192093743218


 53%|█████▎    | 132/250 [03:31<03:08,  1.60s/it]

Epoch 133, Train loss: 10.927998044755723


KeyboardInterrupt: 

In [59]:
test_model(model, test_loader, device)

Test MAE: 11.541644399128263
MAPE for high targets: 0.03323933627042505
MAE for low targets: 4.533527544566563


In [58]:
import torch

def test_model(model, test_loader, device):
    model.to(device)  # S'assurer que le modèle est sur le bon device
    model.eval()  # Mode évaluation (désactive dropout, batchnorm...)

    loss_mae = 0
    loss_mape = 0
    MAPE_high_target = 0
    MAE_low_target = 0

    MAE = CustomMAELoss()
    MAPE = CustomMAPELoss()

    with torch.no_grad():  # Désactivation des gradients pour économiser la mémoire
        for graph, temporal_features in test_loader:
            graph = graph.to(device)  # Déplacer le graphe sur GPU
            temporal_features = temporal_features.to(device, non_blocking=True).squeeze()
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)
            target = graph.y.to(device, non_blocking=True)  # Déplacer target sur GPU

            output = model(graph, temporal_features)  # Prédiction sur GPU

            loss_mae += MAE(output, target).item()
            loss_mape += MAPE(output, target).item()

            MAPE_high_target_, MAE_low_target_ = get_flow_forecasting_metrics(output, target)
            MAPE_high_target += float(MAPE_high_target_)
            MAE_low_target += float(MAE_low_target_)

    print(f"Test MAE: {loss_mae / len(test_loader)}")
    print(f"MAPE for high targets: {MAPE_high_target / len(test_loader)}")
    print(f"MAE for low targets: {MAE_low_target / len(test_loader)}")


### Test 2 GPU batch size = 32

In [65]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 250

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        temporal_features = temporal_features.to(device, non_blocking=True)
        temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            temporal_features = temporal_features.to(device, non_blocking=True)
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  0%|          | 1/250 [00:00<03:32,  1.17it/s]

Epoch 1, Train loss: 54.08898611236037
Epoch 1, Test loss: 23.367042422294617


  1%|          | 2/250 [00:01<03:29,  1.18it/s]

Epoch 2, Train loss: 21.53876729597125
Epoch 2, Test loss: 19.114429712295532


...

Epoch 248, Train loss: 11.17767365773519
Epoch 248, Test loss: 11.832853496074677


100%|█████████▉| 249/250 [03:35<00:00,  1.24it/s]

Epoch 249, Train loss: 11.236089773345412
Epoch 249, Test loss: 11.799924612045288
Epoch 250, Train loss: 11.181410990263286


100%|██████████| 250/250 [03:36<00:00,  1.15it/s]

Epoch 250, Test loss: 11.821921348571777


In [66]:
test_model(model, test_loader, device)

Test MAE: 11.799169361591339
MAPE for high targets: 0.03375766787212342
MAE for low targets: 4.686810076236725


In [67]:
test_model(model, train_loader, device)

Test MAE: 11.202487460353918
MAPE for high targets: 0.03386963897368364
MAE for low targets: 4.492502153965464


### Test 3 GPU batch size = 16

In [68]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8, batch_size=16, shuffle=True)

In [69]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 250

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        temporal_features = temporal_features.to(device, non_blocking=True)
        temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            temporal_features = temporal_features.to(device, non_blocking=True)
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  0%|          | 1/250 [00:01<04:22,  1.05s/it]

Epoch 1, Train loss: 43.299754817928886
Epoch 1, Test loss: 20.443081945180893


  1%|          | 2/250 [00:02<04:18,  1.04s/it]

Epoch 2, Train loss: 18.82448738233178
Epoch 2, Test loss: 28.519316732883453


...

Epoch 248, Train loss: 11.073030594175895
Epoch 248, Test loss: 11.671331524848938


100%|█████████▉| 249/250 [04:41<00:01,  1.03s/it]

Epoch 249, Train loss: 11.047512202136284
Epoch 249, Test loss: 11.62993386387825
Epoch 250, Train loss: 11.061775870027795


100%|██████████| 250/250 [04:42<00:00,  1.13s/it]

Epoch 250, Test loss: 11.687492698431015


In [70]:
test_model(model, test_loader, device)

Test MAE: 11.708517402410507
MAPE for high targets: 0.03346586041152477
MAE for low targets: 4.606855109333992


In [71]:
test_model(model, train_loader, device)

Test MAE: 11.065855427125914
MAPE for high targets: 0.03339383930825027
MAE for low targets: 4.377037879640022


### Test GPU without temporal features, batch size = 16 (to compare with the last cell)

In [72]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + node_features, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)

        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

In [73]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 250

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        # temporal_features = temporal_features.to(device, non_blocking=True)
        # temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            # temporal_features = temporal_features.to(device, non_blocking=True)
            # temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  0%|          | 1/250 [00:01<04:14,  1.02s/it]

Epoch 1, Train loss: 47.26280099311761
Epoch 1, Test loss: 19.156573593616486


  1%|          | 2/250 [00:02<04:11,  1.01s/it]

Epoch 2, Train loss: 19.577083351337805
Epoch 2, Test loss: 24.19652372598648


...

Epoch 248, Train loss: 11.312954619922468
Epoch 248, Test loss: 11.918468028306961


100%|█████████▉| 249/250 [04:43<00:01,  1.07s/it]

Epoch 249, Train loss: 11.296911636285023
Epoch 249, Test loss: 11.931018680334091


100%|██████████| 250/250 [04:44<00:00,  1.14s/it]

Epoch 250, Train loss: 11.293305409693085
Epoch 250, Test loss: 11.879538238048553


In [78]:
test_model(model, test_loader, device)

Test MAE: 11.927453517913818
MAPE for high targets: 0.034039315360132605
MAE for low targets: 4.717480376362801


In [79]:
test_model(model, train_loader, device)

Test MAE: 11.285873117700087
MAPE for high targets: 0.03407631882062528
MAE for low targets: 4.498197236947254


## Window Size comparaison

In [96]:
class ImprovedSTGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, temporal_features_dim=10):
        super(ImprovedSTGCN, self).__init__()
        self.gcn1 = GCNConv(node_features, hidden_dim)  # GCN
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + node_features + temporal_features_dim, hidden_dim)  # Prédiction finale
        self.fc2 = nn.Linear(hidden_dim + node_features, 1)
        # self.bn1 = nn.BatchNorm1d(hidden_dim)

    def forward(self, graph, temporal_features):
        x = self.gcn1(graph.x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)
        x = self.gcn2(x, graph.edge_index)  # GCN
        x = F.relu(x) # (num_nodes, hideen_dim)

        # Use temporal features and x as inputs of fc and use the fc layer
        x = torch.cat((x, graph.x, temporal_features), dim=-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = torch.cat((x, graph.x), dim=-1)
        x = self.fc2(x)
        x = F.relu(x)
        return x # return the flow for each nodes (num_nodes, 1)

### window size = 2

In [97]:
window_size = 2

In [98]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8, batch_size=16)

In [99]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 150

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        temporal_features = temporal_features.to(device, non_blocking=True)
        temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            temporal_features = temporal_features.to(device, non_blocking=True)
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  1%|          | 1/150 [00:01<02:55,  1.17s/it]

Epoch 1, Train loss: 39.00233799835731
Epoch 1, Test loss: 19.83356218627005


  1%|▏         | 2/150 [00:02<02:44,  1.11s/it]

Epoch 2, Train loss: 17.74400205447756
Epoch 2, Test loss: 19.88889286734841


...

Epoch 148, Train loss: 13.827018030758563
Epoch 148, Test loss: 14.790377934773764


 99%|█████████▉| 149/150 [02:39<00:01,  1.10s/it]

Epoch 149, Train loss: 13.84526567623533
Epoch 149, Test loss: 14.849621252580123


100%|██████████| 150/150 [02:40<00:00,  1.07s/it]

Epoch 150, Train loss: 13.809438072401901
Epoch 150, Test loss: 14.723855972290039


In [100]:
test_model(model, test_loader, device)

Test MAE: 14.734387715657553
MAPE for high targets: 0.043267088518901306
MAE for low targets: 6.181398203878691


In [101]:
test_model(model, train_loader, device)

Test MAE: 13.825291502064672
MAPE for high targets: 0.043417384623196616
MAE for low targets: 5.658682634090555


### window size = 8

In [102]:
window_size = 8

In [103]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8, batch_size=16)

In [104]:
import torch
from tqdm import tqdm

# Détection du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisation du modèle et des composants d'entraînement
model = ImprovedSTGCN(node_features=window_size, hidden_dim=32).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.008)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.L1Loss()  # MAE

num_epochs = 150

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0

    for graph, temporal_features in train_loader:
        graph = graph.to(device)  # Déplacement du graphe sur GPU
        temporal_features = temporal_features.to(device, non_blocking=True)
        temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y.to(device, non_blocking=True)  # Déplacement de la cible sur GPU

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            graph = graph.to(device)
            temporal_features = temporal_features.to(device, non_blocking=True)
            temporal_features = temporal_features.flatten(start_dim=0, end_dim=1)

            output = model(graph, temporal_features)
            target = graph.y.to(device, non_blocking=True)

            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")


Using device: cuda


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 1, Train loss: 67.7957610816599


  1%|          | 1/150 [00:01<03:39,  1.47s/it]

Epoch 1, Test loss: 21.957280985514323
Epoch 2, Train loss: 17.97388901665946


...

Epoch 148, Test loss: 9.962190087636312
Epoch 149, Train loss: 9.40896459160564


 99%|█████████▉| 149/150 [02:52<00:01,  1.23s/it]

Epoch 149, Test loss: 9.989104652404786
Epoch 150, Train loss: 9.433659910041595


100%|██████████| 150/150 [02:53<00:00,  1.15s/it]

Epoch 150, Test loss: 9.962934255599976


In [107]:
test_model(model, test_loader, device)

Test MAE: 9.973845783869425
MAPE for high targets: 0.027982774501045544
MAE for low targets: 3.8746155977249144


In [108]:
test_model(model, train_loader, device)

Test MAE: 9.447987605478161
MAPE for high targets: 0.02773873460975206
MAE for low targets: 3.8119984965458094


**Window_size=8:**

Test MAE: 9.973845783869425

MAPE for high targets: 0.027982774501045544

MAE for low targets: 3.8746155977249144


**Window_size=4:**

Test MAE: 11.927453517913818

MAPE for high targets: 0.034039315360132605

MAE for low targets: 4.717480376362801


**Window_size=2:**

Test MAE: 14.734387715657553

MAPE for high targets: 0.043267088518901306

MAE for low targets: 6.181398203878691